# Troubleshooting Authentication with DefaultAzureCredential

## Overview

DefaultAzureCredential is the recommended way to handle authentication in Azure applications. It provides a streamlined authentication flow by trying multiple credential types in sequence until one succeeds. This notebook will help you troubleshoot common authentication issues and ensure proper setup.

## Understanding DefaultAzureCredential

DefaultAzureCredential attempts authentication methods in the following order:

1. Environment Credentials
2. Workload Identity (in Kubernetes)
3. Managed Identity
4. Azure CLI Credentials
5. Azure PowerShell Credentials
6. Visual Studio Code Credentials
7. Interactive Browser Authentication (as fallback)

## Prerequisites

Ensure you have the following installed:
- Azure CLI
- Azure Developer CLI (optional)
- Python Virtual Environment or Conda (use `uv venv` or `conda create`)
- Required role assignments (Azure AI Developer)
- Jupyter Notebook environment - kernel configured to use Python 3.8 or later

## Authentication Methods

### 1. Using Azure CLI (Recommended for Local Development)

In [ ]:
# Install required packages
!pip install azure-identity

# First, we'll authenticate using Azure CLI
This is the recommended approach for local development.

When you run the code below, you will be redirected to:
- Either the Azure portal in your browser to complete the login 
- Or use Windows login if you're already signed in to your machine

The code will:
1. Load environment variables from cred.json file, including the TENANT_ID
2. Use Azure CLI to log in to your specific tenant  
3. Test authentication by attempting to get a token


In [ ]:
import os
import json

def find_file(filename, search_path):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# Search in the parent directory and its subdirectories
parent_dir = os.path.dirname(os.getcwd())  # Get parent directory
file_path = find_file('cred.json', parent_dir)

print(f"Found cred.json at: {file_path}")

try:
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)
    
    # Print the values to verify
    print("Project Connection String:", loaded_config['PROJECT_CONNECTION_STRING'])
    print("Tenant ID:", loaded_config['TENANT_ID'])
    
except FileNotFoundError:
    print(f"Could not find file at: {file_path}")
except json.JSONDecodeError:
    print(f"File exists but contains invalid JSON")


In [ ]:
# Import required packages
from IPython.display import display
from IPython.display import HTML
import getpass
from dotenv import load_dotenv
import os
from pathlib import Path  # For cross-platform path handling
import json
print(loaded_config)

print(f"Tenant ID: {loaded_config["TENANT_ID"]}")
tenant_id = loaded_config["TENANT_ID"]
subscription_id = loaded_config["PROJECT_CONNECTION_STRING"].split(';')[1]

# Azure login with specific tenant
!az login --tenant {tenant_id}

# Get subscription ID from connection string
conn_str = loaded_config["PROJECT_CONNECTION_STRING"]
print(f"Connection string: {conn_str}")
# subscription_id = conn_str.split(';')[1] if conn_str else None

if subscription_id:
    # Set the subscription
    !az account set --subscription {subscription_id}
    print(f"✓ Successfully set subscription: {subscription_id}")
else:
    print("⚠️ Could not get subscription ID from PROJECT_CONNECTION_STRING")


# Next, we'll test the authentication by attempting to get a token using DefaultAzureCredential

DefaultAzureCredential will try multiple authentication methods in this order:
1. Environment credentials (if environment variables are set)
2. Managed Identity credentials (if running in Azure)
3. Shared Token Cache credentials (from previous logins) 
4. Visual Studio Code credentials (if using VS Code)
5. Azure CLI credentials (which we just set up)

The code below will:
1. Create a DefaultAzureCredential instance
2. Try to get a token for Azure Cognitive Services
3. Print success message if token is acquired

>Note: You may see some warning/error messages as it tries different authentication methods - 
>this is normal and can be ignored as long as you see "Successfully acquired token!" at the end


In [ ]:
# Then use DefaultAzureCredential in your code
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
import logging

# Enable detailed logging
logging.basicConfig(level=logging.DEBUG)

try:
    credential = DefaultAzureCredential()
    # Test token acquisition
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully acquired token!")
except Exception as e:
    print(f"Authentication failed: {str(e)}")